In [1]:
!pip install torch transformers accelerate pdfplumber python-docx

  Using cached python_docx-1.2.0-py3-none-any.whl.metadata (2.0 kB)
Using cached python_docx-1.2.0-py3-none-any.whl (252 kB)
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.0 MB 6.7 MB/s eta 0:00:01
   ------------------ --------------------- 1.8/4.0 MB 4.6 MB/s eta 0:00:01
   -------------------------- ------------- 2.6/4.0 MB 4.0 MB/s eta 0:00:01
   ------------------------------- -------- 3.1/4.0 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 3.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pdfplumber
import docx

c:\Users\pranav.vetkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from huggingface_hub import hf_hub_download
from huggingface_hub import snapshot_download
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

model_id = "microsoft/Phi-3-mini-4k-instruct"
local_dir = "./models/phi3-mini"

snapshot_download(
    repo_id=model_id,
    local_dir=local_dir,
    allow_patterns=[
        "config.json",
        "tokenizer.json", "tokenizer_config.json", "vocab.json", "merges.txt", "*.model",
        "*.safetensors", "*.bin", "*.pt"
    ],
    resume_download=True,
    ignore_patterns=["*.md", "*.zip", "*.pdf", "README*"]
)

c:\Users\pranav.vetkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\pranav.vetkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 6 files: 100%|██████████| 6/6 [05:03<00:00, 50.67s/it] 


'C:\\workspace\\Usecases\\Loan Document Summarizer – Banking\\models\\phi3-mini'

In [4]:
from huggingface_hub import snapshot_download
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

model_id = "microsoft/Phi-3-mini-4k-instruct"
local_dir = "./models/phi3-mini"

snapshot_download(
    repo_id=model_id,
    local_dir=local_dir,
    allow_patterns=[
        "configuration*.py",
        "modeling*.py",
        "generation_config.json",
        "tokenizer_config.json",
        "special_tokens_map.json",
        "preprocessor_config.json",
        "*.py",
        "*.json"
    ],
    resume_download=True,
)
print("Done. Check local dir for python files.")

c:\Users\pranav.vetkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 10 files: 100%|██████████| 10/10 [00:03<00:00,  2.91it/s]

Done. Check local dir for python files.


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_dir = "./models/phi3-mini"

print("Tokenizer from:", model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

print("Loading model (this may take a minute). Using low_cpu_mem_usage=True...")
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto" if torch.cuda.is_available() else None,
    torch_dtype="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

if not torch.cuda.is_available():
    model.to("cpu")

print("Model loaded successfully!")

Tokenizer from: ./models/phi3-mini
Loading model (this may take a minute). Using low_cpu_mem_usage=True...


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.34it/s]

Model loaded successfully!


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pdfplumber
import docx

In [7]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    return_full_text=False,
    do_sample=False,
    temperature=0.7,
)

print("Pipeline created successfully.")

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Pipeline created successfully.


In [8]:
def read_file(file_path):
    if not os.path.exists(file_path):
        return None

    text = ""
    try:
        if file_path.endswith('.pdf'):
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    text += page.extract_text() + "\n"
        elif file_path.endswith('.docx'):
            doc = docx.Document(file_path)
            text = "\n".join([p.text for p in doc.paragraphs])
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
        
    return text

def chunk_text(text, chunk_size=2500, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

print("Helper functions defined.")

Helper functions defined.


In [13]:
def extract_loan_info_manual(text_chunk):
    system_prompt = "You are a strict banking assistant. Extract the specific loan details requested below."
    user_prompt = f"""
    Analyze the loan document text provided below and extract these specific fields.
    Return the answer as a bulleted list. If a detail is missing, write "Not Found".

    FIELDS TO EXTRACT:
    - Borrower Name
    - Lender Name
    - Loan Amount
    - Interest Rate
    - Tenure
    - Property Address
    - Penalty Clause

    DOCUMENT TEXT:
    {text_chunk}
    """
    
    full_prompt = f"<|user|>\n{system_prompt}\n{user_prompt}<|end|>\n<|assistant|>"
    device = "cpu"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=500,
            do_sample=False,
            temperature=0.3,
            use_cache=False
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if "<|assistant|>" in result:
        return result.split("<|assistant|>")[-1].strip()
    else:
        return result.replace(full_prompt, "").strip()

print("Manual extraction logic defined (Bug Fix Applied).")

Manual extraction logic defined (Bug Fix Applied).


In [14]:
# --- MAIN EXECUTION (FIXED) ---

filename = "LoanDoc1.pdf"
print(f"Reading {filename}...")
raw_text = read_file(filename)

if not raw_text:
    print("⚠️ PDF not found. Using mock data.")
    raw_text = """
    HOME LOAN AGREEMENT
    BETWEEN PV INVESTMENTS LIMITED (Lender) AND MR. CHAITANYA THAKRE (Borrower)
    LOAN REFERENCE NO: PVI-HL-2025-8892
    1.1 "Loan Amount": ₹75,00,000 (Seventy-Five Lakhs)
    1.2 "Interest Rate": 8.50% per annum (Floating)
    1.3 "Tenure": 240 Months (20 Years)
    1.4 "Property": Unit 702, Tower C, Blue Ridge Township, Hinjewadi, Pune.
    5.1 Default: Penalty interest of 24% per annum.
    """

chunks = chunk_text(raw_text)
print(f"Document split into {len(chunks)} chunk(s).")

print("Analyzing document (Direct Model Call)...")
# Call the new manual function
summary = extract_loan_info_manual(chunks[0])

print("\n" + "="*40)
print("      🏦 LOAN DOCUMENT SUMMARY      ")
print("="*40)
print(summary)
print("="*40)

Reading LoanDoc1.pdf...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
You are not running the flash-attention implementation, expect numerical differences.


Document split into 3 chunk(s).
Analyzing document (Direct Model Call)...


KeyboardInterrupt: 

In [ ]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct" 

print(f"⏳ Loading {model_id} (Fast CPU Mode)...")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype="auto", 
    device_map="cpu",
    trust_remote_code=True
)
print("✅ Model loaded!")

def extract_fast(text_chunk):
    prompt = f"""<|im_start|>system
You are a banking assistant. Extract the loan details from the text below.
Format: Bullet points. If missing, say 'N/A'.
<|im_end|>
<|im_start|>user
Text:
{text_chunk}

Required Details:
- Borrower Name
- Lender Name
- Loan Amount
- Interest Rate
- Tenure
- Property Address
<|im_end|>
<|im_start|>assistant
"""
    
    inputs = tokenizer(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=200,
            do_sample=False,
            temperature=0.0,
            use_cache=False 
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("assistant\n")[-1]

filename = "LoanDoc1.pdf"

# Read File
print(f"\n📖 Reading {filename}...")
full_text = ""
if os.path.exists(filename):
    with pdfplumber.open(filename) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"
else:
    print("⚠️ File not found, using Mock Data.")
    full_text = "HOME LOAN AGREEMENT... Loan Amount: ₹75,00,000... Interest: 8.5%..."

print("🚀 Analyzing (this will take < 30 seconds)...")
summary = extract_fast(full_text[:2000])

print("\n" + "="*40)
print("      🏦 FAST LOAN SUMMARY      ")
print("="*40)
print(summary.strip())
print("="*40)

⏳ Loading Qwen/Qwen2.5-0.5B-Instruct (Fast CPU Mode)...


c:\Users\pranav.vetkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pranav.vetkar\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


✅ Model loaded!

📖 Reading LoanDoc1.pdf...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🚀 Analyzing (this will take < 30 seconds)...

      🏦 FAST LOAN SUMMARY      
- Borrower Name: CHAITANYA THAKRE
- Lender Name: PV INVESTMENTS LIMITED
- Loan Amount: ₹75,00,000
- Interest Rate: 8.50%
- Tenure: 240 Months (20 Years)
- Property Address: Unit 702, Tower C, Blue Ridge Township, Hinjewadi, Pune
